# Full Demo Video - Stiching it all together

In this tutorial we will do a review of everything we have learned by building a video that shows how you can use wallaroo to improve your computer vision ai results.

In [ ]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils

from CVDemoUtils import CVDemo



### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [ ]:
# The size the image will be resized to
width = 640
height = 480

# set the device we will be using to run the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The set of COCO classifications
CLASSES = pickle.loads(open("models/coco_classes.pickle", "rb").read())

# Unique colors for each identified COCO class
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# The size the image will be resized to
width = 640
height = 480
fps = 25
# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo()


### Choose the video we want to perform object detection on

Next lets choose the video we want to perform object detection on and explain
to the user what we are about to do.

In [ ]:

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-4-intro-cv.mp4"


startFrame = 0
endFrame = 100
padding = 10
sentPadding = 15

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : fps, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["Discover how Wallaroo can help","improve your computer vision results."],
    'dashboard-start-frame' : startFrame + padding,
    'dashboard-end-frame' : endFrame - padding,
    'color':CVDemo.WHITE,
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)


In [ ]:

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-4-mobilenet-intro.mp4"

duration = 100
startFrame = endFrame
endFrame = startFrame + duration

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : fps, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["Lets start by deploying a pipeline with a","single stage mobilenet object detector."],
    'dashboard-start-frame' : startFrame + padding,
    'dashboard-end-frame' : endFrame - padding,
    'color':CVDemo.WHITE,
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)



### Deploy the mobilenet Pipeline 

Next we will upload the mobilenet model to wallaroo, tell wallaroo what resources we need, and create the mobilenet pipeline

In [ ]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import pandas as pd

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
model_name = 'mobilenet'
onnx_model_path = 'models/mobilenet.pt.onnx'
mobilenet_model = wl.upload_model(model_name, onnx_model_path)

Next we will add our post processing anomoly detection file called post-process-anomoly-detection.py.  Predictions that are lower than 75% we will consider anomalies that need to be inspected.

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("12Gi").build()

In [ ]:
pipeline_name = 'mobilenet-pp'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .deploy(deployment_config = deployment_config)


In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

In [ ]:

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-4-mobilenet-inferenced.mp4"

#startFrame = endFrame
startFrame = endFrame

noteStart = startFrame + 60
endFrame = noteStart + 410

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : fps, # Frames per second
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'color':CVDemo.CYAN, # color to draw bounding boxes and the text in the statistics
    'inference' : 'WALLAROO_SDK', # "ONNX" or "WALLAROO_API" or "WALLAROO_SDK"
    'onnx_model_path' : onnx_model_path,
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'deployment_config' : deployment_config,
    'pipeline_name' : 'mobilenet-pp',
    'wl' : wl,
    'model' : mobilenet_model,
    'pipeline_name' : pipeline_name,
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video  
    'note-list' : [ { 'note' : ' The Cyan color rectangles are the mobilenet object detections.',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart,
                      'end-frame': noteStart + 100},
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                      { 'note' : 'The object classifications are located above the bounding box',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 150,
                      'end-frame': noteStart + 250  },
                     { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 250,
                      'end-frame': noteStart + 300  },
                       { 'note' : ' and the % confidence in classifications to the right of it.',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 300,
                      'end-frame': noteStart + 400 }],
    'skip-frames-list' : [ (442,460) ]
                      
}

cvDemo.DEBUG = False
cvDemo.detectAndClassifyObjectsInVideo(config)
print("We are done.")


In [ ]:
pipeline.undeploy()

In [ ]:


input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-4-resnet-intro.mp4"


duration = 100
#startFrame = endFrame
startFrame = 670

endFrame = startFrame + duration

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : fps, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : startFrame, # the # of frames to capture in the output video    
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["Now, lets deploy a challenger. ", "The resnet50 object detector."],
    'dashboard-start-frame' : startFrame + padding ,
    'dashboard-end-frame' : endFrame - padding ,
    'color':CVDemo.WHITE,
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)



    

### Deploy the resnet50 Pipeline

Next we will upload the mobilenet model to wallaroo, tell wallaroo what resources we need, and create the mobilenet pipeline

In [ ]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import pandas as pd

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
model_name = 'resnet50'
onnx_model_path = 'models/frcnn-resnet.pt.onnx'
resnet_model = wl.upload_model(model_name, onnx_model_path)

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("12Gi").build()

In [ ]:
pipeline_name = 'resnet-pp'
pipeline = wl.build_pipeline('resnet-pp') \
            .add_model_step(resnet_model) \
            .deploy(deployment_config = deployment_config)

In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

In [ ]:
#pipeline.undeploy()


In [ ]:
input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-4-resnet-inferenced.mp4"

#startFrame = endFrame
startFrame = 670

noteStart = startFrame + 60
endFrame = noteStart + 410

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : fps, # Frames per second
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'color':CVDemo.ORANGE, # color to draw bounding boxes and the text in the statistics
    'inference' : 'WALLAROO_SDK', # "ONNX" or "WALLAROO_API" or "WALLAROO_SDK"
    'onnx_model_path' : onnx_model_path,
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'pipeline_name' : pipeline_name,
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video  
    'note-list' : [ { 'note' : ' Notice, the resnet50 model is able to detect more objects. ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart,
                      'end-frame': noteStart + 80 },
                      { 'note' : 'This is seen in the [Obj] and [Cls] columns.',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 90,
                      'end-frame': noteStart + 130 },
                       { 'note' : 'but inferences [Inf] are significantly slower. ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 140,
                      'end-frame': noteStart + 190 }],
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyObjectsInVideo(config)
print("We are done.")



In [ ]:
pipeline.undeploy()

### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

Next we will load each frame in the input-video feedubg ut to the pipeline for inferencing.  Then using the results we will draw a bounding box around each identified object, print its classification, and the model's confidence that the prediction is accurate on the frame and save the frame to an output video.

As we are executing this notice the time it takes to process each frame.  In the next section we will discuss ways to improve this performance.

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
control =  wl.upload_model('mobilenet', 'models/mobilenet.pt.onnx')

In [ ]:
challenger_list  = [ 
    wl.upload_model('resnet50', 'models/frcnn-resnet.pt.onnx')
]

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("12Gi").build()

In [ ]:
pipeline = wl.build_pipeline("shadow-pp")
pipeline.add_shadow_deploy(control, challenger_list)
pipeline.deploy(deployment_config = deployment_config)

In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

In [ ]:
padding = 10
duration = 100
startFrame = 1139
endFrame = startFrame + duration

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-5-shadow-part-1-intro.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["Lets compare the models using a pipeline", 
                                "that has been configured for shadow deployment.",
                                "The control is mobilenet, the challenge is resnet50"],
    'dashboard-start-frame' : startFrame + padding ,
    'dashboard-end-frame' : endFrame - padding ,
    'color':CVDemo.WHITE,
}


cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

cvDemo.DEBUG = False
cvDemo.recordVideo(config)



    

## Start Here


In [ ]:

startFrame = endFrame
noteStart = startFrame + 60
endFrame = noteStart + 475

# Only objects that have a confidence > confidence_target will be displayed on the image
input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-5-shadow-part-1-inferenced.mp4"

cur_ws = wl.get_current_workspace()
config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 25, # Frames per second
    'pipeline' : pipeline,
    'control-model' : control,
    'challenger-model-list' : challenger_list,
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'inference': 'WALLAROO_SDK',
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video  
    'note-list' : [ { 'note' : ' The Cyan color rectangles are the mobilenet object detections',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart,
                      'end-frame': noteStart + 100},
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                     { 'note' : 'The Orange color rectangles are the mobilenet object detections',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 150,
                      'end-frame': noteStart + 250 },
                     { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 250,
                      'end-frame': noteStart + 300  },
                      { 'note' : 'A great visulization for the data scientist. ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 300,
                      'end-frame': noteStart + 350 },
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 350,
                      'end-frame': noteStart + 375  },
                    { 'note' : 'Enabling data scientists to determine which model is better',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 375,
                      'end-frame': noteStart + 475 }
                    ],
        'skip-frames-list' : [ (1400,1420) ]                  
}

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

cvDemo.detectAndClassifyObjectsInVideoUsingShadowDeployment(config)
print("We are done.")


### Using Shadow Deployment to have the best of both worlds

Now we will run inferences again using shadow deployment, but on each frame we will inspect the inferences
for both the control and challenger models and choose the one that has the best average score above 75%

In [ ]:


input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-shadow-part-2-intro.mp4"

duration = 125
startFrame = endFrame
endFrame = startFrame + duration

cvDemo.print("startFrame:"+str(startFrame))
cvDemo.print("endFrame:"+str(endFrame))

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 25, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["What if we could have the best of both worlds?", 
                                "The next we will use the shadow deployment pipeline to ", 
                                "help us choose the best model."],
    'dashboard-start-frame' : startFrame + padding ,
    'dashboard-end-frame' : endFrame - padding ,
    'color':CVDemo.WHITE,
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)



    

In [ ]:

startFrame = endFrame
noteStart = startFrame + 60
endFrame = noteStart + 475

# Only objects that have a confidence > confidence_target will be displayed on the image
input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-shadow-part-2-inferenced.mp4"

cur_ws = wl.get_current_workspace()
config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 25, # Frames per second
    'pipeline' : pipeline,
    'control-model' : control,
    'challenger-model-list' : challenger_list,
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : 790, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'inference': 'WALLAROO_SDK',
    'record-start-frame' : 791, # the # of frames to capture in the output video
    'record-end-frame' : 890, # the # of frames to capture in the output video  
    'note-list' : [ { 'note' : ' Notice ONLY the mobilenet or resnet50 model bounding boxes is displayed',
                      'color' : CVDemo.CYAN,
                      'start-frame': noteStart,
                      'end-frame': noteStart + 100},
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                     { 'note' : 'The model with the best classification confidence %',
                      'color' : CVDemo.CYAN,
                      'start-frame': noteStart + 150,
                      'end-frame': noteStart + 250 },
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                      { 'note' : 'This is reflected in the [Wins] column',
                      'color' : CVDemo.CYAN,
                      'start-frame': 807,
                      'end-frame': 811 },
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                      { 'note' : 'The best of both of worlds!',
                      'color' : CVDemo.CYAN,
                      'start-frame': 812,
                      'end-frame': 816 }
                    ]
}
cvDemo.DEBUG = False
cvDemo.useBestObjectDetectorInVideoUsingShadowDeployment(config)
print("We are done.")

 'note-list' : [ { 'note' : ' The Cyan color rectangles are the mobilenet object detections',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart,
                      'end-frame': noteStart + 100},
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 100,
                      'end-frame': noteStart + 150  },
                     { 'note' : 'The Orange color rectangles are the mobilenet object detections',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 150,
                      'end-frame': noteStart + 250 },
                     { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 250,
                      'end-frame': noteStart + 300  },
                      { 'note' : 'A great visulization for the data scientist. ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 300,
                      'end-frame': noteStart + 350 },
                    { 'note' : ' ',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 350,
                      'end-frame': noteStart + 375  },
                    { 'note' : 'Enabling data scientists to determine which model is better',
                      'color' : CVDemo.MAGENTA,
                      'start-frame': noteStart + 375,
                      'end-frame': noteStart + 475 }
                    ]

In [ ]:
pipeline.undeploy()

### Model Insights - Anomalies

Wallaroo is a wonderful platform to help you gain insights into the potential anomalies that maybe occuring in your
production computer vision models.

In the example below we are going to setup a wallaroo custom anomoly that will track and display detected objects with a confidence score below 75%.

These are opportunities for the data scientist to inspect each anomoly to determine if the model can be retrained to include or discard the anomoly detected.

In [ ]:

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-anomalies-intro.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : 890, # the # of frames to capture in the output video
    'record-start-frame' : 891, # the # of frames to capture in the output video
    'record-end-frame' : 970, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["Wallaroo Anomalies helps data scientists quickly  ", 
                                "and easily identify anomalies in object detection models.", 
                                "Let's check it out"],
    'dashboard-start-frame' : 900 ,
    'dashboard-end-frame' : 960 ,
    'color':CVDemo.WHITE
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)



    

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
model_name = 'mobilenet'
mobilenet_model = wl.upload_model('mobilenet', "models/mobilenet.pt.onnx")

Next we will add our post processing anomoly detection file called post-process-anomoly-detection.py.  Predictions that are lower than 75% we will consider anomalies that need to be inspected.

In [ ]:
module_anomoly_detection = wl.upload_model("post-process-anomoly-detection", "./post-process-anomoly-detection.py").configure('python')

### Deploy our custom anomoly detection

Next we will deploy our pipeline with the custom anomoly detection logic as a post processing step in our pipeline.

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("12Gi").build()

In [ ]:
pipeline_name = 'anomoly-pp'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .add_model_step(module_anomoly_detection)

pipeline.deploy(deployment_config = deployment_config)

In [ ]:
url = pipeline._deployment._url()
print(url)


## Test the pipeline in a video stream

### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [ ]:

from CVDemoUtils import CVDemo

# set the device we will be using to run the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The set of COCO classifications
CLASSES = pickle.loads(open("models/coco_classes.pickle", "rb").read())

# Unique colors for each identified COCO class
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo(CLASSES,COLORS, DEVICE)


### Detect and Classify Anomalies in the Video Stream

Next we will load each frame in the input-video feed itto the pipeline for inferencing.  The pipeline will perform normal inferencing logic and then before returning the results execute our custom anomoly detection.  The anomalies detected, meaning their bounding boxes, their classification and confidence of classifications are included in the inferencing results.

We use these results to draw all the anomalies their bounding boxes, each identified object,  its classification, and the model's confidence that the prediction and then save the frame to an output video.


In [ ]:
# The size the image will be resized to
width = 640
height = 480

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-anomalies-inferenced.mp4"


config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : 960, # the # of frames to capture in the output video
    'color':CVDemo.ORANGE, # color to draw bounding boxes and the text in the statistics
     #'inference' : 'WALLAROO_SDK' is only supported for anomalies right now
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'pipeline_name' : pipeline_name,
    'record-start-frame' : 962, # the # of frames to capture in the output video
    'record-end-frame' : 1060, # the # of frames to capture in the output video  
    'note-list' : [ { 'note' : 'Object detections below 75% is an anomaly.',
                      'color' : CVDemo.CYAN,
                      'start-frame': 964,
                      'end-frame': 968},
                     { 'note' : 'Each bounding box identifies an anomoly.',
                      'color' : CVDemo.CYAN,
                      'start-frame': 970,
                      'end-frame': 975 },
                      { 'note' : 'The [Anom] column tells us how many.',
                      'color' : CVDemo.CYAN,
                      'start-frame': 978,
                      'end-frame': 983 },
                     { 'note' : 'Notice the avg conf % [conf] is extremely low.',
                      'color' : CVDemo.CYAN,
                      'start-frame': 985,
                      'end-frame': 990 }
                      { 'note' : 'Great opportunities for model improvement identified.',
                      'color' : CVDemo.CYAN,
                      'start-frame': 985,
                      'end-frame': 990 }
                    ]
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyAnomaliesInVideo(config)
print("We are done.")





In [ ]:
#pipeline.undeploy()

### To learn more message

Lets add a learn more message


In [ ]:
duration = 100
startFrame = endFrame
startFrame = 1780
endFrame = startFrame + duration


input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-step-9-conclusion.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'skip-frames' : startFrame, # the # of frames to capture in the output video
    'record-start-frame' : startFrame, # the # of frames to capture in the output video
    'record-end-frame' : endFrame, # the # of frames to capture in the output video    'skip-frames' : 75, # the # of frames to capture in the output video
    'dashboard-message-list' : ["To learn how to do this with your computer vision ",
                                "models checkout the Wallaroo Tutorials.", 
                                "Thanks for watching!"],
    'dashboard-start-frame' : startFrame + padding ,
    'dashboard-end-frame' : startFrame - padding ,
    'color':CVDemo.WHITE,
}
cvDemo.DEBUG = False
cvDemo.recordVideo(config)


    

### Conclusion

To stitch it all together, we will take each video generated in this tutorial and concatenate them into 1 final video

videos/amazon-fresh-go-final.mp4

That is it!  You now know how to use wallaroo to improve your computer vision projects.  Good luck with your efforts!


In [ ]:

output_video = ""

config = {
    'video-list' : ['videos/amazon-fresh-go-step-4-intro-cv.mp4',
                    'videos/amazon-fresh-go-step-4-mobilenet-intro.mp4',
                    'videos/amazon-fresh-go-step-4-mobilenet-inferenced.mp4',
                    'videos/amazon-fresh-go-step-4-resnet-intro.mp4',
                    'videos/amazon-fresh-go-step-4-resnet-inferenced.mp4',
                    'videos/amazon-fresh-go-step-5-shadow-part-1-intro.mp4',
                    'videos/amazon-fresh-go-step-5-shadow-part-1-inferenced.mp4',
                    #'videos/amazon-fresh-go-shadow-part-2-intro.mp4',
                    #'videos/amazon-fresh-go-shadow-part-2-inferenced.mp4',
                    #'videos/amazon-fresh-go-anomalies-intro.mp4', 
                    #'videos/amazon-fresh-go-anomalies-inferenced.mp4',
                    'videos/amazon-fresh-go-step-9-conclusion.mp4'
                   ], # source video
    'output-video' : 'videos/amazon-fresh-go-step-final.mp4', # record all videos stitched together in this file
    'fps': 15,
    'width' : width, # the width of the videos
    'height' : height, # the height of the videos 
}
cvDemo.DEBUG = False
cvDemo.stichVideosTogether(config)



In [ ]:

output_video = ""

config = {
    'video-list' : [
                    'videos/amazon-fresh-go-shadow-part-1-inferenced.mp4',
                    'videos/amazon-fresh-go-shadow-part-2-intro.mp4',
                  #  'videos/amazon-fresh-go-shadow-part-2-inferenced.mp4',
                   ], # source video
    'output-video' : 'videos/test-shadow.mp4', # record all videos stitched together in this file
    'fps': 15,
    'width' : width, # the width of the videos
    'height' : height, # the height of the videos 
}
cvDemo.DEBUG = False
cvDemo.stichVideosTogether(config)

In [ ]:
for d in wl.list_deployments():
    d.undeploy()
    